<a href="https://colab.research.google.com/github/rudevico/Gachon-AISTUDY/blob/main/PyTorch_2_tensor%2C_storage%2C_stride.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tensor
tensor는 결국 Numpy ndarray라고 생각하면 된다(완전히 동일한 개념은 아니다).  
즉 n-dimensional array라는 것인데, 결국 memory상에서 연산하기 위해서는 1-dimensional의 vector array가 되어야 한다.

# storage
n-dimensional array를 1-dimensional의 vecter array로 변환한 것이 **storage**이다.

In [ ]:
import torch

# 2 by 3의 2-dimensional tensor
tensor = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(tensor.shape)
print(tensor)

# tensor가 memory상에서 연산되기 위해서는 storage로 변환되어야 한다
storage = tensor.storage()
# 이때 storage 자체는 tensor 객체가 아니고, tensor 객체의 데이터가 저장된 array이다
# 따라서 .shape나 .size() 등의 tensor 객체에 대한 메서드 사용 불가
print(f'storage size: {len(storage)}')
# print(storage.shape) # error occured!
print(storage)

torch.Size([2, 3])
tensor([[1, 2, 3],
        [4, 5, 6]])
storage size: 6
 1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]


# stride

In [ ]:
# transposed_tensor
tensor_T = tensor.transpose(0, 1) # convert to 3 by 2 tensor
storage_T = tensor_T.storage()
print(tensor_T.shape)
print(tensor_T)
print(f'storage_T size: {len(storage_T)}')
print(storage_T)

torch.Size([3, 2])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
storage_T size: 6
 1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]


(위 cell)  
`storage`와 `storage_T`는 언뜻 보면 동일하게 보인다.  

하지만 이 둘은 각각 2 by 3의 `tensor`와 3 by 2의 `tensor_T`로부터 비롯된 storage이기 때문에 다른 것이다.  
> 컴퓨터는 이 둘을 어떻게 다른 것으로 구분할까?  
정답은 `stride`에 있다.

In [ ]:
# 결론부터 먼저 확인
print(storage == storage_T) # Output: False
print(tensor.stride(), tensor_T.stride())

False
(3, 1) (1, 3)


그렇다면 `stride`란 무엇일까?  

[torch.Tensor.stride — PyTorch 2.4 documentation](https://pytorch.org/docs/stable/generated/torch.Tensor.stride.html)
> Stride is the jump necessary to go from one element to the next one in the specified dimension dim. A tuple of all strides is returned when no argument is passed in. Otherwise, an integer value is returned as the stride in the particular dimension dim.

이를 풀어서 설명하자면 다음과 같다.  
* * *
**`tensor`의 경우:**  

| tensor | column1 | column2 | column3 |
| :---: | :---: | :---: | :---: |
| row1 | 1 | 2 | 3 |
| row2 | 4 | 5 | 6 |

이를 storage로 나열하면 다음과 같다.  

| offset | 0 | 1 | 2 | 3 | 4 | 5 |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| value | 1 | 2 | 3 | 4 | 5 | 6 |

1. row1, column1의 value인 `1`에서 row2, column1의 value인 `4`로 이동하기 위해서는 `3번의 jump`가 필요하다.  
    * jump1: index 0 → 1
    * jump2: index 1 → 2
    * jump3: index 2 → 3

2. row1, column1의 value인 `1`에서 row1, column2의 value인 `2`로 이동하기 위해서는 `1번의 jump`가 필요하다.  
    * jump1: index 0 → 1

따라서, `tensor.stride()` = (3, 1)

* * *
**`tensor_T`의 경우:**  

| tensor_T | column1 | column2 |
| :---: | :---: | :---: |
| row1 | 1 | 4 |
| row2 | 2 | 5 |
| row3 | 3 | 6 |

이를 storage로 나열하면 다음과 같다.  

| offset | 0 | 1 | 2 | 3 | 4 | 5 |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| value | 1 | 2 | 3 | 4 | 5 | 6 |

1. row1, column1의 value인 `1`에서 row2, column1의 value인 `2`로 이동하기 위해서는 `1번의 jump`가 필요하다.  
    * jump1: index 0 → 1

2. row1, column1의 value인 `1`에서 row1, column2의 value인 `4`로 이동하기 위해서는 `3번의 jump`가 필요하다.  
    * jump1: index 0 → 1
    * jump2: index 1 → 2
    * jump3: index 2 → 3

따라서, `tensor_T.stride()` = (1, 3)

In [30]:
print(tensor.stride(), tensor_T.stride())

(3, 1) (1, 3)
